# Knickpoint Analyses for Geoscience
-Nick Lewis, Joanmarie Del Vecchio, Chuck Bailey.

Steps
- Get Dem (take user bounds, res.)
- Download geology shp files
- Download WBD GDB
- User can now select what they want to do.
Export files include:
	- Folders for each watershed w name
        - Relief map
            - W/ LSHT highlighted
	    - Slope map (similar but not the same!)
            - LSHT
	    - Knickpoints mapped (maybe adjust delta ksn?  To isolate bedrock reaches.)
		- Profile w trimodal morphology represented (maybe, refer to chuck suggestion)
	- CSVs of knickpoints
		- To be included:
        - Wshed
		- elevation
		- Coords
		- Shapefile geology
		- Nearest roadcut geology that is also within some threshold of elevation from knickpoint.  Do buffer around knickpoint select closest to elevation.
		- Stream cover type (to allow narrowing to bedrock)
		- If BR, fracture data, if AP, bedding
		- If not in AP or BR, Send back a message being like sorry dawg.
		- Depth to mantle @ knickpoint.  Z score to average depth in region.
		- Slope

# Code begins here!

#### Below is a 'code cell'.  This is a block of text that Jupyter and Python use to perform tasks.

#### To run the following code cell and all code cells below, select the cell and press
# <p style="color: aqua;">Shift+Enter.</p>

In [1]:
# This is called a comment.  Denoted by the '#' behind the text, it allows me to better explain code cells while still being a begign line of text.

# First, we import the YML file.  This saves you the work of making your own python environment and installing a bunch of packages.
#!conda env create -f env.yml

In [2]:
# From google drive, let's get the peripheral files our code will use.
#function here that imports shit

In [8]:
# Import packages to our notebook.
# We must do this to use the functions contained inside the packages.
# Some functions have 'as' in the import call.  This is for less typing, and nothing else.
# Google any package name and 'docs' in order to find more detailed explanation of any one package.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import xdem
import os
import rasterio as rio
from rasterio.plot import show
import glob
from concurrent.futures import ThreadPoolExecutor
import requests

#### Phew!  That was a lot of initial stuff, but no worries.  We can start *really* coding now.

#### Below is a code cell containing our **Constant** Variables, which you may change as you see fit for your particular analyses.

#### Below are 
- Integers
- Lists (denoted by square brackets at the beginning and end, separated by commas [a, b, c, d])
- Strings (contained within quotes 'Like this!')

In [9]:
# Every time you change something within the code cell, run it once more to update the variable.

BOUNDS = [38.0, -78.5, 37.5, -80.0] # In decimal coordinate form.  Format: [N, E, S, W].  Select the smallest area possible in your analyses for maximum resoultion!  
# For now, keep them in Virginia or West VA.

API_KEY = '9086887f175d33aa72eda767f5b1e9cd' # Get your API key from opentopography.org.  Students get higher resolution data!

DEM_DATASET = 'USGS30m' # Options are USGS30m, USGS10m, & USGS1m.  Be warned!  As resolution goes up, available area goes down.

WBD_SCALE = 8 # Size threshold of watersheds your DEM will be split into.  From largest to smallest, the options are 4, 6, 8, & 10.  See below for examples of scale.

PROJ_TITLE = 'n_lewis' # Create a succinct name with no spaces or leading digits to represent your project file for future exports.

#### If you can't remember what a function does, call the help() function and pass the name of the function you are wondering about inside the colons.

In [10]:
# Downloader
def downloader(bounds:list = BOUNDS, api_key:str = API_KEY, dem_dataset:str = DEM_DATASET, proj_title:str = PROJ_TITLE): # To call a function, we have to define it first.
    """
    This function takes in the user-defined constants and creates a directory.  It then places a geotiff in the directory.

    args: Bounds (list), api_key (str), dem_dataset (str), proj_title (str).

    returns: None.
    """

    # First, we make our directory for project files.
    path = os.getcwd() # Gets your current working directory.
    try:
        os.mkdir(path = f'{path}/{proj_title}')
        print(f'Directory "{path}/{proj_title}" created successfully.') # Tries to make the directory for project files.
        proj_dir = f'{path}/{proj_title}'
    except FileExistsError:
        print(f'Directory "{path}/{proj_title}" already exists.') # If it doesnt work, it'll tell you!
        proj_dir = f'{path}/{proj_title}'
    except Exception as e:
        print(f'An error occurred: {e}')

    # Now to download our dem.
    dic = dict(zip(['north', 'east', 'south', 'west'], bounds)) # Makes a dictionary to let us more easily acess our bounds.
    url = f'https://portal.opentopography.org/API/usgsdem?datasetName={dem_dataset}&south={dic["south"]}&north={dic["north"]}&west={dic["west"]}\
&east={dic["east"]}&outputFormat=GTiff&API_Key={api_key}' # Creates our URL so we can download.
    print('Querying API...')
    response = requests.get(url) # Saves what the URL spit back at us
    print('URL passed, now waiting for file to write.')
    with open(f'{proj_dir}/gtiff', 'wb') as file:
        file.write(response.content) # Writes the response to our project file
    print('File Written.')

downloader() # Here, we call the function.

Directory "/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/jupyter_proto/n_lewis" already exists.
Querying API...
URL passed, now waiting for file to write.
File Written.


load df- Make CSVs for each scale so that the wait time is no so stupid.\
Get wsheds in dem bounds\
extract geometries and names,\
export dems w names that are just those wsheds.\
Also create reprojected versions.\
Set nodata vals to -99999\

In [ ]:
# Watershed divider
wsheds = gpd.read_file('/sciclone/home/ntlewis/Nick-Lewis-Research/working_files/data/WBD_National_GDB/WBD_National_GDB.gdb', layers=['WBDHU4'])

In [14]:
wsheds[wsheds.states.isin(['VA', 'WV', 'NC', 'TN'])]

,tnmid,metasourceid,sourcedatadesc,sourceoriginator,sourcefeatureid,loaddate,referencegnis_ids,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontributingareaacres,noncontributingareasqkm,globalid,shape_Length,shape_Area,geometry
191,{E0E694E5-0012-4FEB-8C04-BC6A974AFD92},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,1465282,10753.19,43.52,VA,...,Couches Creek,S,NM,030102040204,0.0,0.0,{B2A2DF37-E29C-11E2-8094-0021280458E6},0.333907,0.004405,"MULTIPOLYGON (((-78.27523 37.02693, -78.27484 ..."
287,{BC75B77F-95EE-45C0-91E9-D373EA3D6BAD},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,1538644,25688.66,103.96,WV,...,Headwaters Elkhorn Creek,S,NM,050702010203,0.0,0.0,{AAFE631D-E29C-11E2-8094-0021280458E6},0.534342,0.010579,"MULTIPOLYGON (((-81.39507 37.46196, -81.39492 ..."
299,{0432273F-E9E0-4FC4-80AA-9ADDB0A272EC},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,"1483220,1484600",26436.85,106.99,VA,...,East Wilderness Creek-Kimberling Creek,S,NM,050500020105,0.0,0.0,{B33FB818-E29C-11E2-8094-0021280458E6},0.532044,0.010856,"MULTIPOLYGON (((-81.03335 37.23799, -81.03275 ..."
301,{7CDF4478-6998-4CA2-BF5F-CE2D257E8373},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,1485878,15636.90,63.28,VA,...,Nobusiness Creek,S,NM,050500020105,0.0,0.0,{B33FD0B5-E29C-11E2-8094-0021280458E6},0.454672,0.006425,"MULTIPOLYGON (((-80.83373 37.26347, -80.83418 ..."
305,{FF9C9CE2-41EA-4FD3-8A66-11A01539F47E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,"1481530,1499795",13538.35,54.79,VA,...,Bear Spring Branch-New River,S,KA,050500020304,0.0,0.0,{B3406080-E29C-11E2-8094-0021280458E6},0.428540,0.005566,"MULTIPOLYGON (((-80.62134 37.30491, -80.62134 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98562,{7667ECDB-1442-45F9-995E-9F0DEF9C537B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,"1478131,1488853",32234.50,130.45,VA,...,Bailey Creek-James River,S,NM,020802060205,0.0,0.0,{B0F69506-E29C-11E2-8094-0021280458E6},0.713788,0.013256,"MULTIPOLYGON (((-77.20652 37.37646, -77.20523 ..."
98563,{C42113E6-776F-4325-BC1D-A5F238A0BAB2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,1480071,41831.81,169.29,VA,...,Lynnhaven River,S,NM,020801080202,0.0,0.0,{B310B856-E29C-11E2-8094-0021280458E6},0.886414,0.017106,"MULTIPOLYGON (((-76.03466 36.92141, -76.03459 ..."
99513,{957EA744-3B11-49D1-A2D6-F07908983569},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,"1478853,206699",14569.45,58.96,VA,...,Jericho Ditch-Dismal Swamp,D,"DD,IT",030102050604,0.0,0.0,{B2C99CB2-E29C-11E2-8094-0021280458E6},0.335827,0.005943,"MULTIPOLYGON (((-76.55952 36.68294, -76.55650 ..."
99714,{31823352-27A0-4C82-A434-12E56E5ED363},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15 06:55:45+00:00,"1478508,206699",40240.18,162.85,VA,...,Lake Drummond-Dismal Swamp,D,AW,030102050606,0.0,0.0,{B2C9B386-E29C-11E2-8094-0021280458E6},0.841272,0.016406,"MULTIPOLYGON (((-76.46409 36.76978, -76.46349 ..."


In [1]:
# Relief & LSHT toolbox


In [ ]:
# KSN toolbox

In [ ]:
# Mapper

In [ ]:
# DF Maker

In [ ]:
# Exporter